<a href="https://colab.research.google.com/github/EdwynZN/TLGColab/blob/main/SC23_Reto_ZambranoNemegyei_Edwin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import os

def flatMultiIndex(data: pd.DataFrame) -> pd.DataFrame:
    data.columns = data.columns.get_level_values(1)
    return data.reset_index()

def excelWriter(path: str) -> pd.ExcelWriter:
    if os.path.exists(path):
        return pd.ExcelWriter(path, if_sheet_exists='replace', mode='a')
    else:
        return pd.ExcelWriter(path)

# Get only relevant columns
employeeR1 = pd.read_csv('sample_data/SC23_data/test.csv', usecols=['Employee_ID', 'Age', 'Education_Level'], dtype={'Age': 'Int64', 'Education_Level' : 'Int64'})
employeeR2 = pd.read_csv('sample_data/SC23_data/train.csv', usecols=['Employee_ID', 'Age', 'Education_Level'], dtype={'Age': 'Int64', 'Education_Level' : 'Int64'})
deviceR1 = pd.read_csv('sample_data/SC23_data/test2.csv', sep='\s*,\s*', engine='python')
deviceR2 = pd.read_csv('sample_data/SC23_data/train2.csv', sep='\s*,\s*', engine='python')

employees = pd.concat([employeeR1, employeeR2], ignore_index=True)
devices = pd.concat([deviceR1, deviceR2], ignore_index=True)

singleDF = pd.merge(employees, devices, on='Employee_ID')
# Get only relevant columns
col = ['Computer_OS', 'Computer_OS_version', 'Mobile_OS', 'Mobile_OS_version']
singleDF[col] = singleDF.loc[:, col].fillna('No registrado')

# Total numbers of rows
count = singleDF.shape[0]
print('N° de filas', count)

groupOS = singleDF.groupby('Computer_OS')
groupMobileOS = singleDF.groupby('Mobile_OS')

# Porcentaje de personas que usan cada sistema operativo en su computadora
computerDF = groupOS.agg({'Computer_OS': ['count']})
computerDF = flatMultiIndex(computerDF)
computerDF.rename(columns = {'count':'Total'}, inplace = True)
computerDF['Porcentaje'] = computerDF.Total * 100.0 / count
print('computer OS', computerDF, sep='\n')

# Porcentaje de personas que usan cada sistema operativo en su dispositivo móvil
mobileDF = groupMobileOS.agg({'Mobile_OS': ['count']})
mobileDF = flatMultiIndex(mobileDF)
mobileDF.rename(columns = {'count':'Total'}, inplace = True)
mobileDF['Porcentaje'] = mobileDF.Total * 100.0 / count
print('Mobile OS', mobileDF, sep='\n')

# Edad promedio (suma de todas las edades de cada grupo / el total de personas)
ageDF = groupOS.agg({'Age': ['mean']})
ageDF = flatMultiIndex(ageDF)
ageDF.rename(columns = {'mean':'Edad_Promedio'}, inplace = True)
# No existen edades con decimales, truncando para entender mejor al momento de leer
ageDF['Edad_Promedio'] = ageDF['Edad_Promedio'].astype('Int64')
print('Edades promedio:', ageDF, sep='\n')

# Nivel educativo promedio de la gente que utiliza cada sistema operativo
educationLevelDF = groupOS.agg({'Education_Level': ['mean']})
educationLevelDF = flatMultiIndex(educationLevelDF)
educationLevelDF.rename(columns = {'mean':'Nivel_Educativo_Promedio'}, inplace = True)
print('Nivel educativo promedio:', educationLevelDF, sep='\n')

# N° total de tickets generados por la gente en cada sistema operativo
computerTicketsDF = groupOS.agg({'Computer_tickets': ['sum']})
computerTicketsDF = flatMultiIndex(computerTicketsDF)
computerTicketsDF.rename(columns = {'sum':'Tickets_Generados'}, inplace = True)
print('N° total de tickets:', computerTicketsDF, sep='\n')

# Guardar nuevos DataFrames.
fileName = 'sample_data/SC23_data/SC23_Reto_ZambranoNemegyei_Edwin.xlsx'
with excelWriter(fileName) as writer:
    singleDF.to_excel(writer, sheet_name="Base de datos")
    computerDF.to_excel(writer, sheet_name="Computer_OS_Percent")
    mobileDF.to_excel(writer, sheet_name="Mobile_OS_Percent")
    ageDF.to_excel(writer, sheet_name="Computer_OS_Mean_Age")
    educationLevelDF.to_excel(writer, sheet_name="Computer_OS_Education_Level")
    computerTicketsDF.to_excel(writer, sheet_name="Computer_OS_Total_Tickets")


N° de filas 10000
computer OS
     Computer_OS  Total  Porcentaje
0          Linux    480        4.80
1          MacOS   1634       16.34
2  No registrado    158        1.58
3        Windows   7728       77.28
Mobile OS
       Mobile_OS  Total  Porcentaje
0        Android   7659       76.59
1  No registrado    287        2.87
2            iOS   2054       20.54
Edades promedio:
     Computer_OS  Edad_Promedio
0          Linux             40
1          MacOS             40
2  No registrado             40
3        Windows             39
Nivel educativo promedio:
     Computer_OS  Nivel_Educativo_Promedio
0          Linux                   3.24375
1          MacOS                  3.168299
2  No registrado                  3.177215
3        Windows                  3.203675
N° total de tickets:
     Computer_OS  Tickets_Generados
0          Linux                254
1          MacOS               1880
2  No registrado                541
3        Windows              21169
